## Title: Intact Forest Landscapes (2016)

### Description
The Intact Forest Landscapes (IFL) data set identifies unbroken expanses of natural ecosystems within the zone of forest extent that show no signs of significant human activity and are large enough that all native biodiversity, including viable populations of wide-ranging species, could be maintained. To map IFL areas, a set of criteria was developed and designed to be globally applicable and easily replicable, the latter to allow for repeated assessments over time as well as verification. IFL areas were defined as unfragmented landscapes, at least 50,000 hectares in size, and with a minimum width of 10 kilometers. These were then mapped from Landsat satellite imagery for the year 2000.<br>

Changes in the extent of IFLs were identified from 2000-2013 and from 2013-2016 within the original year 2000 IFL boundary using the global wall-to-wall Landsat image composite for years 2013, 2016, and the global forest cover loss dataset (Hansen et al., 2013). Areas identified as “reduction in extent” met the IFL criteria in 2000, but no longer met the criteria in 2016. The main causes of change were clearing for agriculture and tree plantations, industrial activity such as logging and mining, fragmentation due to infrastructure and new roads, and fires assumed to be caused by humans.<br>

This data can be used to assess forest intactness, alteration, and degradation at global and regional scales.<br>

The world's IFL map is a spatial database (scale 1:1,000,000) that shows the extent of the intact forest landscapes (IFL) for years 2000, 2013, and 2016.

### FLINT
This dataset has been pre-processed/checked and is suitable for use in FLINT. Please adhere to individual dataset licence conditions and citations. Processed data can be accessed here: https://datasets.mojaglobal.workers.dev/

### Format
<b>Extent: </b>Global coverage<br>
<b>Format</b>: polygon geoJSON .json<br>
<b>Cordinate system:</b> EPSG:4326 (WGS84)<br>
<b>Temporal Resolution: </b>2000, 2013, 2016<br>
<b>Size:</b> 1.86gb <br>

### Original source
Original Source: http://www.intactforests.org/data.ifl.html Accessed 13/12/2020 <br>
Cooridnate system: WGS_1984 (EPSG:4326)

### Licence
Limitations of use: [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/)<br>
Provided IFL data are available for use for valid scientific, conservation, and educational purposes as long as proper citations are used. We ask that you credit the IFL data as follows.

### Citation
Potapov, P., Hansen, M. C., Laestadius L., Turubanova S., Yaroshenko A., Thies C., Smith W.,
Zhuravleva I., Komarova A., Minnemeyer S., Esipova E. 2016. “The last frontiers of wilderness: Tracking
loss of intact forest landscapes from 2000 to 2013” Science Advances, 2017; 3:e1600821

### Original format
Global coverage, shapefile class<br>
Cordinate system EPSG: 4326 (WGS84)

### Metadata
[Data description](http://www.intactforests.org/shp/IFL_2018_readme.pdf)
IFL for year 2000                                   (141 MB)<br>
IFL for year 2013                                   (141 MB)<br>
IFL for year 2016                                   (205 MB)<br>
IFL change 2000-2013-2016                           (344 MB)<br>
IFL change factors 2000-2013                        (212 MB)<br>
Forest zone extent                                  (34 MB)

### Notes
There is some distortion over southern South Ameria where boundaries do not align with coastline. Seems to be fine elsewhere (Japan, Europe, North America, Tasmania, Australia) so is probably a localised error in the original data.<br>

Geometry includes non-simple geometry and duplicate vertices that requires repair.

Datasets "ifl_2000" and "ifl_2016" have overlaps. And all datasets have valid gaps and non-valid gaps for repair.

File sizes are large due to detail and resultion of the vectors. Gaps and overlaps are evident and can be fixed with code below. Processing time is lengthy. Please note, only gaps smaller than 0.5ha will be fixed as gaps larger than this could be valid. Gaps smaller than 0.5ha could also be valid but are removed in this instance for improved drawing speed.

### Processing
Repair geometry, fix topologial error (remove overlaps and gaps smaller than 0.5ha), convert to geojson, EPSG:4326 (WGS84), remove/disable Z values. View code below - originally processed in ArcGIS but can be converted to open source QGIS or GDAL (or others).

In [ ]:
# Import arcpy module
import arcpy
import os

# Input variables (input your own folders here)
in_folder = r"C:\Downloads\IFL"
scr_folder = r"C:\Data\scratch.gdb"
out_folder = r"C:\Data\json"
field = "CLASS_NAME IS NULL OR CLASS_NAME = ''" #unique field any that will show up as blank and thus as an appended gap during fix

# Environments
workspace = in_folder
arcpy.env.workspace = workspace
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4326)
arcpy.env.outputZFlag = "Disabled"
arcpy.env.overwriteOutput = True
scr = arcpy.CreateFileGDB_management(r"C:\Data", "scratch")
arcpy.env.parallelProcessingFactor = "100%"

# List features to process
featureclasses = arcpy.ListFeatureClasses()
print(featureclasses)

# Repair/check topology and make FLINT ready
for fc in featureclasses:
    fcname = os.path.join(os.path.splitext(fc)[0])
    outjson = os.path.join(out_folder, fcname)
    whereclause = "FID_" + fcname + " AND AREA_GEO <= 5000 Or AREA_GEO IS NULL"
    print(fcname + ' processing...')
    
    fLayer = "project_Layer"
    arcpy.management.MakeFeatureLayer(fc, fLayer)
    projectIntersect = os.path.join(scr_folder, "projectIntersect")
    arcpy.analysis.Intersect(fLayer, projectIntersect, "ONLY_FID")
    
    projectSingle = os.path.join(scr_folder, "projectSingle")
    arcpy.management.MultipartToSinglepart(projectIntersect, projectSingle)

    dissolveSlither = os.path.join(scr_folder, "dissolveSlither")
    arcpy.management.Dissolve(projectSingle, dissolveSlither, None, None,"SINGLE_PART")
    
    # Take action if overlaps
    if arcpy.management.GetCount(dissolveSlither)[0] == "0":
        print('no overlaps detected')
        
        projectUnion = os.path.join(scr_folder, "projectUnion")
        arcpy.analysis.Union(fLayer,projectUnion, "ALL", None, "NO_GAPS")
        arcpy.management.AddGeometryAttributes("projectUnion", "AREA_GEODESIC", None, "SQUARE_METERS")

        uniSelect = os.path.join(scr_folder, "uniSelect")
        arcpy.analysis.Select(projectUnion, uniSelect, whereclause)
        
        # Take action if gaps
        if arcpy.management.GetCount(uniSelect)[0] == "0":
                     
            # Progress report no error
            print(fcname, 'No gaps and overlaps. Repairing geometry and conversion to json...')
    
            # Process: Repair Geometry (non-simple geometry)
            geomRepair = arcpy.management.RepairGeometry(fLayer, "DELETE_NULL", "OGC")[0]

            # Process: Features To JSON
            arcpy.conversion.FeaturesToJSON(fLayer, outjson, "NOT_FORMATTED", "NO_Z_VALUES", "NO_M_VALUES", "GEOJSON", "WGS84", "USE_FIELD_NAME")

            print(outjson, '.geojson complete')
            
        else:
            # Take action if gaps
            print('gaps detected')

            appendGap = arcpy.management.Append(uniSelect, fLayer, "NO_TEST")     
            selectGap = arcpy.management.SelectLayerByAttribute(fLayer, "NEW_SELECTION", field)

            fixedlyr = os.path.join(scr_folder, "fixedlyr")
            arcpy.management.Eliminate(selectGap, fixedlyr, "LENGTH")

            # Progress report 
            print(fcname, 'No overlaps, gaps detected and repaired. Repairing geometry and conversion to json...')

            # Process: Repair Geometry (non-simple geometry)
            geomRepair = arcpy.management.RepairGeometry(fixedlyr, "DELETE_NULL", "OGC")[0]

            # Process: Features To JSON
            arcpy.conversion.FeaturesToJSON(fixedlyr, outjson, "NOT_FORMATTED", "NO_Z_VALUES", "NO_M_VALUES", "GEOJSON", "WGS84", "USE_FIELD_NAME")


    else:
        # Fix overlaps
        projectErase = os.path.join(scr_folder, "projectErase")
        arcpy.analysis.Erase(fLayer, dissolveSlither, projectErase)
        arcpy.management.Append(dissolveSlither, projectErase, "NO_TEST")
    
        selectSlither = arcpy.management.SelectLayerByAttribute(projectErase, "NEW_SELECTION",  field)
        
        eliminateSlither = os.path.join(scr_folder, "eliminateSlither")
        arcpy.management.Eliminate(selectSlither, eliminateSlither, "LENGTH")
        
        print('overlaps detected and fixed')
        
        projectUnion = os.path.join(scr_folder, "projectUnion")
        arcpy.analysis.Union(eliminateSlither, projectUnion, "ALL", None, "NO_GAPS")
        arcpy.management.AddGeometryAttributes(projectUnion, "AREA_GEODESIC", None, "SQUARE_METERS")
        
        uniSelect = os.path.join(scr_folder, "uniSelect")
        
        arcpy.analysis.Select(projectUnion, uniSelect, "FID_eliminateSlither = -1 AND AREA_GEO <=5000 OR AREA_GEO IS NULL")
        
        # Take action if no gaps
        if arcpy.management.GetCount(uniSelect)[0] == "0":
            
            # Progress report no error
            print(fcname, 'Overlaps detected and repaired. No gaps detected. Repairing geometry and conversion to json...')
    
            # Process: Repair Geometry (non-simple geometry)
            geomRepair = arcpy.management.RepairGeometry(eliminateSlither, "DELETE_NULL", "OGC")[0]

            # Process: Features To JSON
            arcpy.conversion.FeaturesToJSON(eliminateSlither, outjson, "NOT_FORMATTED", "NO_Z_VALUES", "NO_M_VALUES", "GEOJSON", "WGS84", "USE_FIELD_NAME")

            print(outjson, '.geojson complete')
            
        else:
            # Take action if gaps
            appendGap = arcpy.management.Append(uniSelect, eliminateSlither, "NO_TEST")
            
            selectGap = arcpy.management.SelectLayerByAttribute(eliminateSlither, "NEW_SELECTION",  field)
            
            fixedlyr = os.path.join(scr_folder, "fixedlyr")
            
            arcpy.management.Eliminate(selectGap, fixedlyr, "LENGTH")
            
            print('gaps detected and repaired')
            
            # Progress report 
            print(fcname, 'Gaps and overlaps fixed. Repairing geometry and conversion to json...')
            
            # Process: Repair Geometry (non-simple geometry)
            geomRepair = arcpy.management.RepairGeometry(fixedlyr, "DELETE_NULL", "OGC")[0]

            # Process: Features To JSON
            arcpy.conversion.FeaturesToJSON(fixedlyr, outjson, "NOT_FORMATTED", "NO_Z_VALUES", "NO_M_VALUES", "GEOJSON", "WGS84", "USE_FIELD_NAME")
            
arcpy.AddMessage("All done!")
print('done')